In [18]:
import os
import json
import copy
import random

In [19]:
#data_f = os.path.join("..", "data")
#data_f = os.path.join("C:/", "Users", "kgk", "OneDrive - Aalborg Universitet", "CALDISS_projects", "2021-22_tagging-coercion", "data")
data_f = os.path.join("D:/", "OneDrive", "OneDrive - Aalborg Universitet", "CALDISS_projects", "2021-22_tagging-coercion", "data")
data_n = "tc_anno-ents_comb.jsonl"
data_p = os.path.join(data_f, data_n)

In [8]:
with open(data_p, 'r') as f:
    data = [json.loads(jline) for jline in f.read().splitlines()]

## Isolating labels

The code below isolates specific labels. A dataset for each label is created.

In [63]:
# Possible span labels: CITY, NAME, ADDRESS
span_labels = ['CITY', 'NAME', 'ADDRESS']

datasets = {}

for label in span_labels:
    data_copy = copy.deepcopy(data)
    for entry in data_copy:
        entry['spans'] = [span for span in entry.get('spans') if span.get('label') == label]
    
    datasets[label] = data_copy

In [64]:
filenames = {}
for label in span_labels:
    filename = f"tc_{label}.json"
    filenames[label] = filename

In [65]:
for label in span_labels:
    data_out = os.path.join(data_f, filenames[label])
    with open(data_out, 'w') as outfile:
        json.dump(datasets[label], outfile)

## Combining data

In [20]:
filenames = [filename for filename in os.listdir(data_f) if ".jsonl" in filename]
filenames.remove("tc_pos-nouns.jsonl") # Remove pos tagged data as texts are split

In [21]:
all_data = []

for filename in filenames:
    data_p = os.path.join(data_f, filename)
    
    with open(data_p, 'r') as f:
        data = [json.loads(jline) for jline in f.read().splitlines()]
        
    all_data = all_data + data


In [22]:
ids_wanted = list(set([entry.get('ID') for entry in all_data]))
ids_statements = list(set([entry.get('Id') for entry in all_data]))

all_ids = list(filter(None, ids_wanted + ids_statements))

In [23]:
len(all_ids)

629

In [24]:
unique_entries = []
ids_check = list(filter(None, ids_wanted + ids_statements))
keep_keys = ["ID", "Id", "set", "Navn_anmelder", "Navn_efterlyst", "Date", "text"]

for entry in all_data:
    if (entry.get('ID') in ids_check) or (entry.get('Id') in ids_check):
        entry_id = entry.get('ID')
        if entry_id is None:
            entry_id = entry.get('Id')
        
        entry_filtered = {keep_key: entry.get(keep_key) for keep_key in keep_keys}
        
        unique_entries.append(entry_filtered)
        ids_check.remove(entry_id)
        
for entry in unique_entries:
    if entry.get('Navn_efterlyst') is not None and entry.get('set') is None:
        entry['set'] = "Bortløbningsannoncer"

In [25]:
## Shuffle

random.shuffle(unique_entries)

In [26]:
out_n = "tc_annotate-set-all-mixed_20220902.json"
out_p = os.path.join(data_f, out_n)

# Export data
with open(out_p, 'w', encoding = 'utf-8') as f:
    json.dump(unique_entries, f, ensure_ascii=False)